In [1]:
import numpy as np
import os
from PIL import Image, ImageDraw, ImageFilter
import matplotlib.pyplot as plt
import albumentations as A
import cv2

In [2]:
folder_path = "../../data/raw"
save_dir = '../../data/processed'

transform_shape = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.3, rotate_limit=360, p=0.6, border_mode=cv2.BORDER_CONSTANT),
    A.Perspective(scale=(0, 0.9), keep_size=True, p=0.6),
    A.ElasticTransform(approximate=True, border_mode=cv2.BORDER_CONSTANT, p=0.6),
    A.GridDistortion(border_mode=cv2.BORDER_CONSTANT, p=0.6), 
])
transform_color = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.7),
    A.Blur(blur_limit=50, p=0.7),
    A.GaussNoise(var_limit=50, p=0.9), 
])

os.makedirs(save_dir, exist_ok=True)

In [ ]:
def get_agumentations(folder_path, save_dir, num_samples=300):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
        
            file_path = os.path.join(root, file) 
            image = Image.open(file_path)
            save_subdir = os.path.join(save_dir, file[:-4])
            os.makedirs(save_subdir, exist_ok=True)
        
            for i in range(num_samples): 
            
                mask = Image.new("L", image.size, 0)
                draw = ImageDraw.Draw(mask)
                background = Image.new("L", image.size, 120)
                if i<150: draw.ellipse((25, 25, 110, 110), fill=255) #половина изобрадений с круглой маской, половина - с квадратной
                else: draw.rectangle((30, 30, 110, 110), fill=255)
                blurred_mask = mask.filter(ImageFilter.GaussianBlur(10))
                image_with_mask = Image.new("L", image.size)
                image_with_mask.paste(background, (0, 0))
                image_with_mask.paste(image, (0, 0), mask=blurred_mask)
                image_arr = np.asarray(image_with_mask)
                rotated_image = transform_shape(image=image_arr)['image']
                rotated_image = Image.fromarray(rotated_image)
                result_image = Image.new("L", background.size, 120)  
                result_image.paste(rotated_image, (0, 0), rotated_image)  

                result_image_arr = np.asarray(result_image)
                agumented_image = transform_color(image=result_image_arr)['image']
            
                save_path = os.path.join(save_subdir, f'{i}.jpeg')
                Image.fromarray(np.uint8(agumented_image)).convert("L").save(save_path)

In [ ]:
get_agumentations(folder_path, save_dir)

In [ ]:
# дополняем датасет реальными изображениями интерференционных картин
folder_path_extentions = "../../data/extentions" 
get_agumentations(folder_path_extentions, save_dir)